# Waternet Preprocessing

-----

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
import os
import glob
import laspy
from numba import jit
import numba
import numpy as np
import pandas as pd
from shapely import wkt
import pathlib
import re

import utils.clip_utils as clip_utils
import utils.las_utils as las_utils
import utils.plot_utils as plot_utils

In [ ]:
def get_bbox(points, padding=0):
    """
    Get the <X,Y> bounding box for a given CycloMedia laz file, based on the
    filename.

    Parameters
    ----------
    laz_file : Path or str
        the .laz filename, e.g. filtered_2386_9702.laz
    padding : float
        Optional padding (in m) by which the bounding box will be extended.

    Returns
    -------
    tuple of tuples
        Bounding box with inverted y-axis: (x_min, y_min, x_max, y_max)
    """
    
    (x_min, y_min, x_max, y_max) = compute_bounding_box(points)
    
    return (x_min - padding, y_min - padding, x_max + padding, y_max + padding)

@jit(nopython=True, cache=True)
def rectangle_clip(points, rect):
    """
    Clip all points within a rectangle.

    Parameters
    ----------
    points : array of shape (n_points, 2)
        The points.
    rect : tuple of floats
        (x_min, y_min, x_max, y_max)

    Returns
    -------
    A boolean mask with True entries for all points within the rectangle.
    """
    clip_mask = np.where(((points[:,0] >= rect[0]) & (points[:,0] <= rect[2])
                 & (points[:,1] >= rect[1]) & (points[:,1] <= rect[3])))[0]
    return clip_mask

@jit(nopython=True, cache=True)
def circle_clip(points, center, radius):
    """
    Clip all points within a circle (or unbounded cylinder).

    Parameters
    ----------
    points : array of shape (n_points, 2)
        The points.
    center : tuple of floats (x, y)
        Center point of the circle.
    radius : float
        Radius of the circle.

    Returns
    -------
    A boolean mask with True entries for all points within the circle.
    """
    clip_mask = (np.power((points[:, 0] - center[0]), 2)
                 + np.power((points[:, 1] - center[1]), 2)
                 <= np.power(radius, 2))
    return clip_mask

@jit(nopython=True, cache=True, parallel=True)
def compute_bounding_box(points):
    """
    Get the min/max values of a point list.

    Parameters
    ----------
    points : array of shape (n_points, 2)
        The (x, y) coordinates of the points. Any further dimensions will be
        ignored.

    Returns
    -------
    tuple
        (x_min, y_min, x_max, y_max)
    """
    x_min = np.min(points[:, 0])
    x_max = np.max(points[:, 0])
    y_min = np.min(points[:, 1])
    y_max = np.max(points[:, 1])

    return (x_min, y_min, x_max, y_max)

def get_tilecode_from_filename(filename):
    """Extract the tile code from a file name."""
    return re.match(r'.*(\d{4}_\d{4}).*', filename)[1]

def get_tilecodes_from_folder(las_folder, las_prefix=''):
    """Get a set of unique tilecodes for the LAS files in a given folder."""
    files = pathlib.Path(las_folder).glob(f'{las_prefix}*.laz')
    tilecodes = set([get_tilecode_from_filename(file.name) for file in files])
    return tilecodes

def bbox_to_points(bbox):
    '''
    Parameters
    ----------
    bbox : tuple of floats
        (x_min, y_min, x_max, y_max)
    '''
    return np.array([[bbox[0],bbox[1]], [bbox[0],bbox[3]], [bbox[2],bbox[1]], [bbox[2],bbox[3]]])

def get_bbox_from_tile_code(tile_code, padding=0, width=50, height=50):
    """
    Get the <X,Y> bounding box for a given tile code. The tile code is assumed
    to represent the lower left corner of the tile.

    Parameters
    ----------
    tile_code : str
        The tile code, e.g. 2386_9702.
    padding : float
        Optional padding (in m) by which the bounding box will be extended.
    width : int (default: 50)
        The width of the tile.
    height : int (default: 50)
        The height of the tile.

    Returns
    -------
    tuple of tuples
        Bounding box with inverted y-axis: (x_min, y_min, x_max, y_max)
    """
    tile_split = tile_code.split('_')

    # The tile code of each tile is defined as
    # 'X-coordinaat/50'_'Y-coordinaat/50'
    x_min = int(tile_split[0]) * 50
    y_min = int(tile_split[1]) * 50

    return (x_min - padding, y_min - padding, x_min + height + padding, y_min + height + padding)

def get_bbox_from_las_file(laz_file, padding=0):
    """
    Get the <X,Y> bounding box for a given CycloMedia laz file, based on the
    filename.

    Parameters
    ----------
    laz_file : Path or str
        the .laz filename, e.g. filtered_2386_9702.laz
    padding : float
        Optional padding (in m) by which the bounding box will be extended.

    Returns
    -------
    tuple of tuples
        Bounding box with inverted y-axis: ((x_min, y_max), (x_max, y_min))
    """
    if type(laz_file) == str:
        laz_file = pathlib.Path(laz_file)
    tile_code = get_tilecode_from_filename(laz_file.name)

    return get_bbox_from_tile_code(tile_code, padding=padding)

In [ ]:
bomen_csv = '../../../datasets/Waternet/output_220527.csv'

df = pd.read_csv(bomen_csv, delimiter=';')
df['geometry'] = df['geometry'].apply(wkt.loads)
df['geom2'] = df['geom2'].apply(wkt.loads)
df.head()

In [ ]:
base_folder = '../../../datasets/Waternet/run1/'
out_folder = '../../../datasets/Waternet/clipped/'

for i, row in df.iterrows():
    name = row['Boom']
    tree_name = '_'.join(name.split(' '))
    tree_loc = str(int(row['x_est'])) + '_' + str(int(row['y_est']))
    out_path = out_folder + 'cyclo_' + tree_loc + '_' + tree_name + '.las'
    if not os.path.exists(out_path):
        print(f"Clipping {name}")
        min_bound = np.hstack(row['geometry'].xy)
        max_bound = np.hstack(row['geom2'].xy)
        rd = np.array(row['RD'])
        tree_center = np.array([row['x_est'],row['y_est']])
        clip_radius = np.linalg.norm(tree_center-min_bound) + 7.5

        points = np.empty((0,3))
        colors = np.empty((0,3))
        for base_path in glob.glob(os.path.join(base_folder, '*.laz')):
            base_bbox = get_bbox_from_las_file(base_path)
            if len(rectangle_clip(np.array([min_bound, max_bound]), base_bbox)) > 0:
                base_cloud = laspy.read(base_path)
                base_points = np.vstack([base_cloud.x, base_cloud.y, base_cloud.z]).T
                base_colors = np.vstack([base_cloud.red, base_cloud.green, base_cloud.blue]).T
                mask = circle_clip(base_points, tree_center, clip_radius)
                points = np.concatenate([points, base_points[mask]])
                colors = np.concatenate([colors, base_colors[mask]])

        if len(points) > 100:
            cloud = laspy.create(file_version="1.2", point_format=3)
            cloud.header.offsets = np.min(points, axis=0)
            cloud.x = points[:, 0]
            cloud.y = points[:, 1]
            cloud.z = points[:, 2]
            cloud.red = colors[:, 0]
            cloud.green = colors[:, 1]
            cloud.blue = colors[:, 2]
            
            
            cloud.write(os.path.join(out_path))
            print(f"\twrote clipped tree to {out_path}.")
